# Lab 8: Spherical harmonics
---

## Submission

Please upload your completed notebook (renamed with your name) to Canvas as usual, **along with any scanned hand-written work if applicable**.

## Introduction

Recall that the Fourier series can be used to decompose a function into a sum of various sine and cosine functions. Below, for example, the square wave (blue line) is being constructed from the sum of the first 200 terms of the Fourier series (red line). 

![Thenub314, CC BY-SA 3.0](https://upload.wikimedia.org/wikipedia/commons/b/bc/Fourier_series_for_square_wave.gif)

Wolfram MathWorld has a nice outline of the Fourier series mathematics for this example [here](https://mathworld.wolfram.com/FourierSeriesSquareWave.html).

Spherical harmonics are a similar concept, but they're used to decompose functions defined on the *surface of a sphere*. This makes them naturally suited for global geophysics problems. Recall from lecture that a function $f(\theta, \phi)$ defined on a sphere can be reconstructed via

$$f(\theta, \phi) = \sum_{n=0}^\infty \sum_{m=0}^n [C_{nm}\cos(m\phi) + S_{nm}\sin(m\phi)]P_{nm}(\cos\theta)\,, \tag{1}$$

where the $C_{nm}$ and $S_{nm}$ coefficients are given by

$$
\left.
\begin{cases}
C_{nm} \\
S_{nm}
\end{cases}
\right\}
= \frac{2n + 1}{4\pi}\int_0^{2\pi}\int_0^\pi f(\theta, \phi) P_{nm}(\cos\theta)
\left.
\begin{cases}
\cos(m\phi) \\
\sin(m\phi)
\end{cases}
\right\}
\sin\theta\,\mathrm{d}\theta\,\mathrm{d}\phi\,. \tag{2}
$$

In these equations, $n$ denotes degree and $m$ denotes order. $P_{nm}(\cos\theta)$ is the associated Legendre function of degree $n$ and order $m$. In this lab you will use spherical harmonics to approximate a simple function, using a case analogous to the Fourier series case shown above. Next lab, we'll reconstruct Earth's gravity field (sounds cool, right?) using the same tools you'll learn in this lab.

## 0. Play around with visualizations of spherical harmonics

Use the provided code below to compute and visualize spherical harmonic functions for a variety of $n$ and $m$, and cosine and sine coefficients $C_{nm}$ and $S_{nm}$. Play around for a bit.

In [ ]:
# Use curl to obtain legendre_schmidt.py, which contains the function legendre_schmidt()
!curl -O -s https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_08/legendre_schmidt.py

import numpy as np
import matplotlib.pyplot as plt

from legendre_schmidt import legendre_schmidt


def spharm(n, m, Cnm, Snm, theta, phi):
    """
    Calculate spherical harmonic function of degree n, order m, with
    coefficients Cnm, Snm, at value(s) (theta, phi).
       theta = colatitude, 0 at north pole, radians
       phi   = longitude, radians

    This function uses the normalization that is standard in geodesy,
    following Kaula (1966). See also Lambeck (1988) Geophysical Geodesy.

    To calculate the value at a specific point, pass scalar values of theta
    and phi. In this case, val returns a scalar value.

    To calculate the value at a grid of points, pass vectors of theta and phi.
    In this case, val returns a matrix of values at the grid defined by the
    vectors theta and phi. The indices for vals(i,j) are:
         i = index on values of theta
         j = index on values of phi

    Author: Jeff Freymueller, Univ. of Alaska, Jan. 2004
            (translated to Python by Liam Toney, Oct. 2020)
    """

    # Calculate the Pnm. Note that the Schmidt normalization is for the associated
    # Legendre functions only: (Pnm, Pnm) = 1. When multiplied by the
    # cos(m*phi) and sin(m*phi) terms, the output of legendre_schmidt() does not result in a
    # fully normalized spherical harmonic. Thus we add a renormalization to
    # give the standard geodetic usage. It seems easiest to calculate the
    # Schmidt normalization and renormalize that. 
    #
    # Renormalize the function in accord with Kaula standard used in geodesy
    # This is the difference between Kaula's fully normalized and the Schmidt
    # normalization.
    #
    # See for example, Hofmann-Wellenhof and Moritz, Physical Geodesy, 2nd ed.,
    #  page 23 (section 1.10)
    
    Pnm = legendre_schmidt(n, np.cos(theta))
    
    renorm = np.sqrt(2 * (n + 1))
    
    # Compute the values of the harmonic at all "gridpoints" as defined above.
    return renorm * np.outer(Pnm[m,:], (Cnm * np.cos(m * phi) + Snm * np.sin(m * phi)))


"""
Example code to call spharm()

This program makes two plots. The first is a rectangular plot of the
selected harmonic (red=positive, blue=negative). The second plot shows
the function mapped onto a sphere.
"""

n = 11
m = 8

Cnm = 1
Snm = 1

# Define a regular grid of theta (colatitude) and phi (longitude)
theta = np.linspace(0, np.pi, 101)
phi = np.linspace(0, 2*np.pi, 201)

# Calculate the spherical harmonic function on the grid
vals = spharm(n, m, Cnm, Snm, theta, phi)

# Figure 1: A rectangular plot

fig, ax = plt.subplots()
# Convert to longitude and latitude in degrees (recall that latitude = 90 - colatitude)
ax.pcolormesh(phi*180/np.pi, 90-theta*180/np.pi, vals, shading='nearest', cmap='coolwarm')
ax.set_title(f'($n$, $m$) = ({n}, {m})')
ax.set_xlabel('Longitude (degrees)')
ax.set_ylabel('Latitude (degrees)')
plt.show()

# Figure 2: Map the harmonic onto a sphere

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
x = np.outer(np.cos(phi), np.sin(theta))
y = np.outer(np.sin(phi), np.sin(theta))
z = np.outer(np.ones(np.size(phi)), np.cos(theta))
norm = plt.Normalize(vmin=vals.min(), vmax=vals.max())
# We need to reverse the order of the rows, so that the northern hemisphere (theta = 0 to pi/2)
# ends up on the top of the sphere; we do this with np.flipud()
ax.plot_surface(
    x, y, z, facecolors=plt.cm.coolwarm(norm(np.flipud(vals.T))), rcount=phi.size, ccount=theta.size, shade=True
)
ax.set_proj_type('ortho')
ax.axis('off')
plt.show()

## 1. Approximating a function using associated Legendre functions

Let $f(\theta, \phi)$ be defined on a sphere such that

$$f(\theta, \phi) = 
\begin{cases}
1 \quad \text{for} \quad 0 \leq \phi \leq \pi \\
0 \quad \text{for} \quad \pi < \phi \leq 2\pi
\end{cases}
$$

1. Calculate the degree 0 and 1 (unnormalized) spherical harmonic coefficients $C_{nm}$ and $S_{nm}$ by hand using equation 2. Note that this will be six (short) calculations, for $C_{00}$, $S_{00}$, $C_{10}$, $S_{10}$, $C_{11}$, and $S_{11}$.
2. Now write a computer program to calculate all of the coefficients for up to degree and order 10, and reconstruct the function. Plot the original function and the reconstructed function. Computing the coefficients requires several steps:
    * A loop over $n$ from 0 to 10.
    * A loop over $m$ from 0 to $n$.
    * Computing the $C_{nm}$ and $S_{nm}$ terms through numerical integration. You can use NumPy's `trapz()` function to integrate numerically using the trapezoid rule.
    * Applying equation 1 to reconstruct $f(\theta, \phi)$.

We've provided you with a function called `legendre_schmidt()` that will do the hard work for you of computing the associated Legendre functions $P_{nm}$. Here is a table of the first few (unnormalized) $P_{nm}(\cos\theta)$, which will help you answer question 1 above:

| **_n_ / _m_** | **_m_ = 0**  | **_m_ = 1**  |
|:-------------:|:------------:|:------------:|
| **_n_ = 0**   | $1$          | —            |
| **_n_ = 1**   | cos $\theta$ | sin $\theta$ |

> **Hints:**
> 1. Use the provided `legendre_schmidt()` function, and make sure you renormalize (see the provided code in `spharm()`).
> 2. The `legendre_schmidt()` function will compute all of the associated Legendre functions for $m=0$ to $n$ with a single call. Therefore, you only need to call this function once for each $n$. Look at the help (type `legendre_schmidt?`) and see how the function stores the return values. Start with some very low degree terms ($n=2$ or $n=3$) and familiarize yourself with the structure of the output arrays.
> 3. For a given $n$ and $m$, if the $P_{nm}$ for various values of $\theta$ are stored in a column vector, and the sin and cos terms for various values of longitude are stored in a row vector, the column vector times the row vector will produce a matrix with all of the appropriate theta and phi combinations. See the code in `spharm()`. You can also do this via `for` loops, but it is much, much slower.